In [ ]:
from datascience import *
import numpy as np
## Normal Distribution
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
# helper methods:

def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)  

def standardize(t):
    """Return a table in which all columns of t are converted to standard units."""
    t_su = Table()
    for label in t.labels:
        t_su = t_su.with_column(label + ' (su)', standard_units(t.column(label)))
    return t_su

def correlation(t, label_x, label_y):
    return np.mean(standard_units(t.column(label_x))*standard_units(t.column(label_y)))

def slope(t, x, y):
    """The slope of the regression line (original units)"""
    r = correlation(t, x, y)
    return r * np.std(t.column(y)) / np.std(t.column(x))

def intercept(t, x, y):
    """The intercept of the regression line (original units)"""
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x))

def fit(t, x, y):
    """Return the predicted y-value for each x-value"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a * t.column(x) + b


## Least Squares

In [ ]:
#little_women = Table.read_table('http://inferentialthinking.com/notebooks/little_women.csv')
little_women = Table.read_table('little_women.csv')

little_women = little_women.move_to_start('Periods')
little_women.show(3)

In [ ]:
little_women.scatter('Periods', 'Characters')

In [ ]:
# What correlation do you expect?

correlation(little_women, 'Periods', 'Characters')

In [ ]:
# helper method

def fit(t, x, y):
    """Return the predicted y-value for each x-value"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a * t.column(x) + b

fitted = fit(little_women, 'Periods', 'Characters')  # array of predictions

lw_with_predictions = little_women.with_column('Linear Prediction', fitted) #table
lw_with_predictions.scatter('Periods')  #plot

In [ ]:
# errors: 

actual = lw_with_predictions.column('Characters')
predicted = lw_with_predictions.column('Linear Prediction')
errors = actual - predicted

lw_with_predictions.with_column("Error", errors)

In [ ]:
#function to draw errors

sample = [[131, 14431], [231, 20558], [392, 40935], [157, 23524]]
def lw_errors(slope, intercept):
    print('Slope of Regression Line:    ', np.round(slope), 'characters per period')
    print('Intercept of Regression Line:', np.round(intercept), 'characters')
    little_women.scatter('Periods', 'Characters')
    xlims = np.array([50, 450])
    plots.plot(xlims, slope * xlims + intercept, lw=2)
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=2)


In [ ]:
#slope, intercept and errors:

lw_reg_slope = slope(little_women, 'Periods', 'Characters')
lw_reg_intercept = intercept(little_women, 'Periods', 'Characters')
lw_errors(lw_reg_slope, lw_reg_intercept)

In [ ]:
# takes any slope, any intercept and redraws the line and errors from the same 4 points

lw_errors(50, 10000)

In [ ]:
lw_errors(-100, 50000)

Which lines are better? The ones with small errors.

Goal: Find the line that minimizes the error.

What exactly will we minimize? 

back to slides for Error in Estimation

In [ ]:
def lw_rmse(slope, intercept):
    lw_errors(slope, intercept)
    x = little_women.column('Periods')
    y = little_women.column('Characters')
    fitted = slope * x + intercept
    mse = np.mean((y - fitted) ** 2)
    print("Root mean squared error:", mse ** 0.5)

In [ ]:
# is the error small or large?
lw_rmse(50, 10000)

In [ ]:
# for comparison

lw_rmse(-100, 50000)

In [ ]:
# close to the regression line
lw_rmse(90, 4000)

In [ ]:
lw_rmse(lw_reg_slope, lw_reg_intercept)

back to slides for Least Squares Line and Numerical Optimization

In [ ]:
# drop the square root

def lw_mse(any_slope, any_intercept):
    x = little_women.column('Periods')
    y = little_women.column('Characters')
    fitted = any_slope*x + any_intercept
    return np.mean((y - fitted) ** 2)

In [ ]:
# What is it going to return?

minimize(lw_mse)

In [ ]:
lw_reg_slope, lw_reg_intercept

In [ ]:
# Discussion question

Image("image_minim.png", width=700, height=150)

## Residuals

In [ ]:
Image("resid.png", width=700, height=150)

In [ ]:
galton = Table.read_table('galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
    )


In [ ]:
# function to calculate residuals

def residual(table, x, y):
    return table.column(y) - fit(table, x, y)


In [ ]:
heights = heights.with_columns(
        'Fitted Value', fit(heights, 'MidParent', 'Child'),
        'Residual', residual(heights, 'MidParent', 'Child')
    )
heights

In [ ]:
def scatter_fit(table, x, y):
    table.scatter(x, y, s=15)
    plots.plot(table.column(x), fit(table, x, y), lw=4, color='gold')
    plots.xlabel(x)
    plots.ylabel(y)
    
scatter_fit(heights, 'MidParent', 'Child')    

In [ ]:
# A residual plot: plotting the residuals against the predictor variable (midparent height)

def residual_plot(table, x, y):
    x_array = table.column(x)
    t = Table().with_columns(
            x, x_array,
            'residuals', residual(table, x, y)
        )
    t.scatter(x, 'residuals', color='r')
    xlims = make_array(min(x_array), max(x_array))
    plots.plot(xlims, make_array(0, 0), color='darkblue', lw=4)
    plots.title('Residual Plot')


In [ ]:
residual_plot(heights, 'MidParent', 'Child')


In [ ]:
# Discussion question

Image("resid_plot.png", width=700, height=150)

## What issues can residual plots detect?

In [ ]:
# Discussion question
Image("dugong.png", width=700, height=150)

In [ ]:
# ages are estimates
dugong = Table.read_table('http://www.statsci.org/data/oz/dugongs.txt')
dugong = dugong.move_to_start('Length')
dugong.show()

In [ ]:
dugong.scatter('Length', 'Age')

In [ ]:
# Length is easy to measure. You know the length, predict the age

correlation(dugong, 'Length', 'Age')

In [ ]:
#helper method

def regression_diagnostic_plots(table, x, y):
    scatter_fit(table, x, y)
    residual_plot(table, x, y)


regression_diagnostic_plots(dugong, 'Length', 'Age')



In [ ]:
Image("17_1.png", width=800, height=400)

In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
regression_diagnostic_plots(hybrid, 'acceleration', 'mpg')


In [ ]:
Image("17_2.png", width=400, height=200)

In [ ]:
# What does it mean? Predictions are not equally accurate for different values of acceleration 

In [ ]:
#Residual Plots are Flat Overall

residual_plot(heights, 'MidParent', 'Child')
correlation(heights, 'MidParent', 'Residual')

In [ ]:
# The Average of the Residuals

round(np.mean(heights.column('Residual')), 10)

In [ ]:
dugong = dugong.with_columns(
        'Fitted Value', fit(dugong, 'Length', 'Age'),
        'Residual', residual(dugong, 'Length', 'Age')
    )
dugong


In [ ]:
residual_plot(dugong, 'Length', 'Age')
correlation(dugong, 'Length', 'Residual')

back to slides for Residual Plots are Flat Overall and Discussion Question